# No SQL Databases - MongoDb

In deze bijhorende notebook gaan we een voorbeeld uitwerken van een Document Based NoSQL database.
Hiervoor maken we gebruik van MongoDb, wat de mogelijkheid aanbied om het lokaal te draaien.
Deze applicatie voor het beheer van een NoSQL database is reeds geinstalleerd en kan gestart worden door het volgende commando in een terminal uit te voeren

    mongod
    
Dit commando start de MongoDb Server. 
Zolang deze applicatie draait kan je met MongoDb connecteren via een shell (niet geinstalleerd) of een api zoals pymongo (is geinstalleerd).

In MongoDb begin je met een te connecteren met een bepaalde database.
Dit doe je door een host en poort te kiezen en de naam van een bepaalde database.
Dit is analoog met hoe je een SQL-database aanspreekt.

MongoDb is een document-based NoSqlDatabase wat betekend dat een database bestaat uit een reeks collecties die elk een aantal documenten bevatten.

In de code hieronder connecteren we met een lokale database "les" waarin we twee collecties gaan gebruiken, namelijk "vakken" en "studenten". 
Deze collecties zijn conceptueel analoog aan de tabellen in een SQL-database.

In [2]:
import pymongo
import json

client = pymongo.MongoClient('localhost', 27017)  # connectie met mongodb - gelijkaardig aan connnectiestring
#db = client["les"]
db = client.les

coll_vakken = db.vakken
coll_studenten = db["studenten"] # collecties en databases worden maar aangemaakt als er documenten toegevoegd worden

Bij bovenstaande code is er echter nog een belangrijke opmerking:
**De database en collecties worden lazy aangemaakt**. 
Dit houdt in dat ze maar aangemaakt worden wanneer ze effectief gebruikt worden (dus wanneer er een document toegevoegd wordt).
Bovenstaande code gaat op dit moment nog geen database en collecties aanmaken.

De documenten in MongoDb kunnen voorgesteld worden als Json formaat. 
In python kunnen dictionaries gebruikt worden om deze documenten voor te stellen, bvb voor een de drie vakken van dit keuzetraject:

In [4]:
datascience = {
    "naam": "Data Science",
    "studiepunten": 5,
    "semester": 1
}

bigdata = {
    "naam": "Big Data",
    "studiepunten": 5,
    "semester": 2
}

machinelearning = {
    "naam": "Machine Learning",
    "studiepunten": 6,
    "semester": 1
}

Deze documenten kunnen toegevoegd worden aan de database door middel van volgende code.

In [5]:
# bij het toevoegen wordt er automatische een _id toegevoegd als identifier indien deze nog niet aanwezig is
# deze kan gebruikt worden als foreign key in andere documenten
# insert_many voor meerdere tegelijkertijd
datascience_id = coll_vakken.insert_one(datascience).inserted_id
bigdata_id = coll_vakken.insert_one(bigdata).inserted_id
machinelearning_id = coll_vakken.insert_one(machinelearning).inserted_id

Dat deze documenten goed zijn toegevoegd en de nodige databases en collecties aangemaakt zijn kan gecontroleerd worden op de volgende manier:

In [11]:
print(client.list_database_names())
print(db.list_collection_names()) # studenten bestaat nog niet want nog geen documenten

from pprint import pprint
# Get 1 of meerdere objecten (query kan in de find staan)
# get all
#for doc in coll_vakken.find():
#    pprint(doc)
# get one (eerste die matcht)
# pprint(coll_vakken.find_one())
# get one with filter filter in accolades, meerdere keys mogelijk
# pprint(coll_vakken.find_one({"_id": bigdata_id}))
# pprint(coll_vakken.find_one({"naam": "Machine Learning"}))
#for doc in coll_vakken.find({"studiepunten":5}):
#    pprint(doc)

['admin', 'config', 'familymanager', 'les', 'local']
['vakken']
{'_id': ObjectId('6461eb12f2351d06e12a3128'),
 'naam': 'Data Science',
 'semester': 1,
 'studiepunten': 5}
{'_id': ObjectId('6461eb12f2351d06e12a312a'),
 'naam': 'Machine Learning',
 'semester': 1,
 'studiepunten': 6}


Om de vakken toe te voegen hebben we documenten 1 voor 1 toegevoegd.
Een andere manier is om met een rij van dictionaries te werken om meerdere documenten tegelijkertijd toe te voegen. 
Dit kan bijvoorbeeld als volgt gedaan worden:

In [12]:
import datetime

students = [{
    "studentennummer": 202001546,
    "naam": "Andy Weir",
    "vakken": [{"naam" : "Data Science", "score": 8}, 
               {"naam" : "Big Data", "score": 10}, 
               {"naam" : "Machine Learning", "score": 12}],
    "geboortedatum": datetime.datetime(2000, 4, 24)
},{
    "studentennummer": 202001548,
    "naam": "Albus Dumbledore",
    "vakken": [{"naam" : "Data Science", "score": 14}, 
               {"naam" : "Big Data", "score": 16}, 
               {"naam" : "Machine Learning", "score": 15}],
    "geboortedatum": datetime.datetime(1800, 4, 24)
},{
    "studentennummer": 202001556,
    "naam": "Frodo Baggings",
    "vakken": [{"naam" : "Data Science", "score": 3}, 
               {"naam" : "Big Data", "score": 5}, 
               {"naam" : "Machine Learning", "score": 4}],
    "geboortedatum": datetime.datetime(1960, 4, 24)
}]

coll_studenten.insert_many(students)

In [13]:
for doc in coll_studenten.find():
    pprint(doc)

{'_id': ObjectId('6461f116f2351d06e12a312b'),
 'geboortedatum': datetime.datetime(2000, 4, 24, 0, 0),
 'naam': 'Andy Weir',
 'studentennummer': 202001546,
 'vakken': [{'naam': 'Data Science', 'score': 8},
            {'naam': 'Big Data', 'score': 10},
            {'naam': 'Machine Learning', 'score': 12}]}
{'_id': ObjectId('6461f116f2351d06e12a312c'),
 'geboortedatum': datetime.datetime(1800, 4, 24, 0, 0),
 'naam': 'Albus Dumbledore',
 'studentennummer': 202001548,
 'vakken': [{'naam': 'Data Science', 'score': 14},
            {'naam': 'Big Data', 'score': 16},
            {'naam': 'Machine Learning', 'score': 15}]}
{'_id': ObjectId('6461f116f2351d06e12a312d'),
 'geboortedatum': datetime.datetime(1960, 4, 24, 0, 0),
 'naam': 'Frodo Baggings',
 'studentennummer': 202001556,
 'vakken': [{'naam': 'Data Science', 'score': 3},
            {'naam': 'Big Data', 'score': 5},
            {'naam': 'Machine Learning', 'score': 4}]}


Om complexere queries uit te voeren moet er gebruik gemaakt worden van de [aggregate functie](https://pymongo.readthedocs.io/en/stable/examples/aggregation.html) waarbij je een stappenplan kan meegeven om een eindresultaat te bekomen.
Meer informatie over alles wat je kan doen met deze aggregate functie kan je vinden in de documentatie van [MongoDb](https://docs.mongodb.com/manual/aggregation/).
Bekijk hiervan zeker de documentatie over [de werking van de pipelines](https://docs.mongodb.com/manual/core/aggregation-pipeline/#std-label-aggregation-pipeline) en de [operators](https://docs.mongodb.com/manual/reference/operator/aggregation/#std-label-aggregation-expression-operators) die je kan gebruiken bij het opstellen van deze pipeline
Nu gaan we een aantal zaken proberen te bereken uit deze data, namelijk:
* Hoeveel vakken zijn er voor elk verschillend aantal studiepunten?
 * Correcte antwoord: 5 studiepunten -> 2 vakken, 6 studiepunten -> 1 vak
* Hoeveel studenten heeft elk vak?
* Voor welke vakken is elke student geslaagd?

In [17]:
pipeline = [
    {"$group": {"_id": "$studiepunten", "naam_kolom": {"$sum": 1}}}
] # dit is een enkelvoudige groupby op de key studiepunten -> aggregatie: maak de som -> tel 1 bij voor elk document
pprint(list(coll_vakken.aggregate(pipeline)))

[{'_id': 6, 'naam_kolom': 1}, {'_id': 5, 'naam_kolom': 2}]


In [19]:
pipeline = [
    {"$unwind": "$vakken"}, # splits de array in individuele documenten
    {"$group": {"_id": "$vakken.naam", "count": {"$sum": 1}}} # selecteer key in subdoc met $key.key2
]
pprint(list(coll_studenten.aggregate(pipeline)))

[{'_id': 'Data Science', 'count': 3},
 {'_id': 'Big Data', 'count': 3},
 {'_id': 'Machine Learning', 'count': 3}]


In [22]:
pipeline = [
    {"$unwind": "$vakken"},
    {"$match": { "vakken.score": { "$gte": 10 } }}, # deze lijn verwijderdt de documenten waar de score van een vak < 10
    {"$group": {"_id": "$naam", "aantal_geslaagd": {"$sum": 1}}}
]
pprint(list(coll_studenten.aggregate(pipeline)))

[{'_id': 'Andy Weir', 'aantal_geslaagd': 2},
 {'_id': 'Albus Dumbledore', 'aantal_geslaagd': 3}]


**Updaten**

Met behulp van de find_one_and_update functie kunnen we gegevens wijzigen.
In de code hieronder gaan we 
* de naam van het vak Data Science wijzigen naar data (en terug)
* het studentennummer met eentje verhogen van Andy Weir
* de score van Andy Weir voor het vak Big Data veranderen naar 20

In [54]:
for doc in coll_vakken.find():
    pprint(doc)
pprint(coll_studenten.find_one({"naam": "Andy Weir"}))

print()
#stap 1
coll_vakken.find_one_and_update({"naam": "Data Science"}, {"$set": {"naam":"data"}})
coll_vakken.find_one_and_update({"naam": "data"}, {"$set": {"naam":"Data Science"}})

print()

#stap 2
from pymongo import ReturnDocument
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$inc": {"studentennummer":1}}, return_document=ReturnDocument.AFTER)
pprint(result)
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$inc": {"studentennummer":-1}}, return_document=ReturnDocument.AFTER)
pprint(result)

print()

#stap 3
arrayFilters = [{
        "vak.naam": "Big Data"
}]
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$set": {"vakken.$[vak].score":20}}, return_document=ReturnDocument.AFTER, array_filters=arrayFilters)
pprint(result)
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$set": {"vakken.$[vak].score":10}}, return_document=ReturnDocument.AFTER, array_filters=arrayFilters)
pprint(result)

# stap 2 en 3
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$inc": {"studentennummer":1}, "$set": {"vakken.$[vak].score":20}}, return_document=ReturnDocument.AFTER, array_filters=arrayFilters)
pprint(result)
result = coll_studenten.find_one_and_update({"naam": "Andy Weir"}, {"$inc": {"studentennummer":-1}, "$set": {"vakken.$[vak].score":10}}, return_document=ReturnDocument.AFTER, array_filters=arrayFilters)
pprint(result)


print()

for doc in coll_vakken.find():
    pprint(doc)

{'_id': ObjectId('6228c015208e54971c8c45c7'),
 'naam': 'Data Science',
 'semester': 1,
 'studiepunten': 5}
{'_id': ObjectId('6228c015208e54971c8c45c8'),
 'naam': 'Big Data',
 'semester': 2,
 'studiepunten': 5}
{'_id': ObjectId('6228c018208e54971c8c45ca'),
 'geboortedatum': datetime.datetime(2000, 4, 24, 0, 0),
 'naam': 'Andy Weir',
 'studentennummer': 202001546,
 'vakken': [{'naam': 'Data Science', 'score': 8},
            {'naam': 'Big Data', 'score': 10},
            {'naam': 'Machine Learning', 'score': 12}]}


{'_id': ObjectId('6228c018208e54971c8c45ca'),
 'geboortedatum': datetime.datetime(2000, 4, 24, 0, 0),
 'naam': 'Andy Weir',
 'studentennummer': 202001547,
 'vakken': [{'naam': 'Data Science', 'score': 8},
            {'naam': 'Big Data', 'score': 10},
            {'naam': 'Machine Learning', 'score': 12}]}
{'_id': ObjectId('6228c018208e54971c8c45ca'),
 'geboortedatum': datetime.datetime(2000, 4, 24, 0, 0),
 'naam': 'Andy Weir',
 'studentennummer': 202001546,
 'vakken': [{'naa

**Verwijderen**

Naast het updaten is het ook mogelijk om verscheidene elementen te verwijderen.
Dit kan aan de hand van een query of door de gewenste collections/databasen te verwijderen.
De code hiervoor is als volgt:

In [51]:
coll_vakken.find_one_and_delete({"naam": "Machine Learning"})
for doc in coll_vakken.find():
    pprint(doc)

{'_id': ObjectId('6228c015208e54971c8c45c7'),
 'naam': 'Data Science',
 'semester': 1,
 'studiepunten': 5}
{'_id': ObjectId('6228c015208e54971c8c45c8'),
 'naam': 'Big Data',
 'semester': 2,
 'studiepunten': 5}


In [27]:
print(db.list_collection_names())
coll_studenten.drop()
print(db.list_collection_names())

['vakken']
['vakken']


In [41]:
print(client.list_database_names())
client.drop_database("les")
print(client.list_database_names())

['admin', 'config', 'les', 'local']
['admin', 'config', 'local']
